<a href="https://colab.research.google.com/github/Humza134/Langchain/blob/main/01_model_prompts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Langchain:
### LangChain is an open-source framework designed to streamline and enhance the process of building applications that use language models (LLMs) like OpenAI's GPT-4, Google Gemini, and similar generative models. It provides a set of tools and abstractions to simplify complex workflows, such as document retrieval, question-answering, and chaining multiple model interactions.

# Install Required Packages

In [ ]:
%%capture --no-stderr
%pip install -U langchain langchain_google_genai
%pip install -U langchain_community

In [ ]:
import os

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/content/feisty-lambda-420416-5d80219f7d8e.json"

## Let's start with a direct API calls to Gemini.

In [ ]:
from google.colab import userdata
GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')

In [ ]:
import google.generativeai as genai
from google.generativeai import GenerativeModel
# set the api key
genai.configure(api_key = GEMINI_API_KEY)

llm_model: GenerativeModel = genai.GenerativeModel('gemini-1.5-flash')

In [ ]:
def completion(prompt:str, model=llm_model)->str:
    """
    Generate a completion for the given prompt using the specified language model.

    Args:
    prompt (str): The input prompt to complete.
    model (ll_model): The language model to use.

    Returns:
    str: The completed text.
    """
    response = model.generate_content(prompt)
    return response.text

In [ ]:
result = completion("What is the capital of Pakistan?")
print(result)

The capital of Pakistan is **Islamabad**. 



## Now let's try how we can do the same thing with langchain

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

# Model Name: specify the model name to use
model_name = "gemini-1.5-flash"
# TEMPERATURE: Control the randomness of model responses (0 = deterministic, 1 = very random)
temperature = 0.0

def chat_completion(prompt:str, model_name=model_name, temperature=temperature)->str:
    """
    Generate a completion for the given prompt using the specified language model.

    Args:
    prompt (str): The input prompt to complete.
    model_name (str): The name of the language model to use.
    temperature (float):
    Control the randomness of model responses (0 = deterministic, 1 = very random).

    Returns:
    str: The completed text.
    """
    llm: ChatGoogleGenerativeAI = ChatGoogleGenerativeAI(
        model=model_name,
        temperature=temperature,
        api_key=GEMINI_API_KEY
        )
    response = llm.invoke(prompt)
    return response

In [ ]:
result = chat_completion("What is the capital of Pakistan?")
print(result)

content='The capital of Pakistan is **Islamabad**. \n' additional_kwargs={} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}]} id='run-9a6fa8ed-e022-49cb-b15a-bc7a95970a87-0' usage_metadata={'input_tokens': 8, 'output_tokens': 9, 'total_tokens': 17, 'input_token_details': {'cache_read': 0}}


In [ ]:
result.content

'The capital of Pakistan is **Islamabad**. \n'

In [ ]:
from langchain_core.messages import SystemMessage, HumanMessage

messages = [
    SystemMessage(content="You are a nice AI bot."),
    HumanMessage(content="What is the capital of Pakistan?"),
]
result = chat_completion(messages)
print(result)


content='The capital of Pakistan is **Islamabad**. \n' additional_kwargs={} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}]} id='run-16f245bd-e3e6-4f56-9bac-270a03c1129b-0' usage_metadata={'input_tokens': 15, 'output_tokens': 9, 'total_tokens': 24, 'input_token_details': {'cache_read': 0}}


# Prompt Template
**A PromptTemplate** in LangChain is a structured way to define and format prompts for language models, enabling dynamic content insertion based on variables. It allows you to create reusable prompts with placeholders that can be customized for various tasks, such as summarization, question-answering, or instruction-following.

## Here are the key features of a PromptTemplate:

**1.** **Dynamic Variable Insertion**:

You can define placeholders  within a prompt **(like {input_variable})** allowing you to customize and reuse templates with different inputs.

**2.Template Standardization**:

PromptTemplate ensures a consistent structure across prompts, improving readability and maintainability.

**3.Ease of Reuse**:

Once created, a template can be reused across multiple chains, tasks, or models, saving time and ensuring prompt consistency.

**4.Error Reduction**:

By defining placeholders in a structured way, it reduces the chance of hardcoded errors and ensures input values are always inserted correctly.

**5.Flexible Prompt Construction**:

You can easily build complex prompts by combining several templates or modifying placeholders based on specific tasks or requirements.

**6.Supports Complex Formatting**:

Advanced templates can be created to support various prompt styles, formatting options, or custom instructions for specific tasks or model behaviors.

In [ ]:
from langchain.prompts import PromptTemplate

# Define a combined template with system instructions and human message
prompt = PromptTemplate(
    input_variables=["question"],
    template=(
        "System: You are an intelligent assistant that provides concise summaries.\n"
        "Human: What is a summary for the following question: {question}?"
    )
)

# Use the template by providing the variable
formatted_prompt = prompt.format(question="What is langchain and what is the purpose of langchain?")
print(formatted_prompt)

result = chat_completion(formatted_prompt)
result

System: You are an intelligent assistant that provides concise summaries.
Human: What is a summary for the following question: What is langchain and what is the purpose of langchain??


AIMessage(content='LangChain is a framework that simplifies building applications powered by large language models (LLMs). Its purpose is to connect LLMs with external data sources and tools, enabling them to perform complex tasks like question answering, summarization, and code generation. \n', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-2f4dc790-cc64-4df7-bafa-5e607b3819cc-0', usage_metadata={'input_tokens': 38, 'output_tokens': 49, 'total_tokens': 87, 'input_token_details': {'cache_read': 0}})

In [ ]:
from langchain_core.prompts import PromptTemplate

template = """In an easy way to translate the following senetence '{sentence}' into {target_language}:"""

prompt = PromptTemplate(
    input_variables=["sentence", "target_language"],
    template=template,
)

formatted_prompt = prompt.format(sentence="How are You?", target_language="Urdu")
print(formatted_prompt)


In an easy way to translate the following senetence 'How are You?' into Urdu:


In [ ]:
result = chat_completion(formatted_prompt)
result

AIMessage(content='The easiest way to translate "How are you?" into Urdu is:\n\n**"Aap kaise hain?" (آپ کیسے ہیں؟)** \n\nThis is the most common and widely understood phrase. \n', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-b17ddd53-1190-4448-bf58-5d3c0650d080-0', usage_metadata={'input_tokens': 20, 'output_tokens': 44, 'total_tokens': 64, 'input_token_details': {'cache_read': 0}})

In [ ]:
result.content

'The easiest way to translate "How are you?" into Urdu is:\n\n**"Aap kaise hain?" (آپ کیسے ہیں؟)** \n\nThis is the most common and widely understood phrase. \n'

In [ ]:
from langchain_core.prompts import PromptTemplate

tempalte = """ create a recipie of {type} cake describe in points."""

prompt = PromptTemplate(
    input_variables=["type"],
    template=tempalte,
)

formatted_prompt = prompt.format(type="chocolate")
print(formatted_prompt)

 create a recipie of chocolate cake describe in points.


In [ ]:
result = chat_completion(formatted_prompt)
result.content

"## Chocolate Cake Recipe:\n\n**Ingredients:**\n\n* **For the Cake:**\n    * 2 cups all-purpose flour\n    * 2 cups granulated sugar\n    * 3/4 cup unsweetened cocoa powder\n    * 1 teaspoon baking soda\n    * 1 teaspoon baking powder\n    * 1/2 teaspoon salt\n    * 1 cup buttermilk\n    * 1/2 cup vegetable oil\n    * 2 large eggs\n    * 1 teaspoon vanilla extract\n    * 1 cup boiling water\n* **For the Frosting:**\n    * 1 cup (2 sticks) unsalted butter, softened\n    * 3 cups powdered sugar\n    * 1/2 cup unsweetened cocoa powder\n    * 1/4 cup milk\n    * 1 teaspoon vanilla extract\n\n**Instructions:**\n\n1. **Preheat oven to 350°F (175°C).** Grease and flour two 9-inch round cake pans.\n2. **Combine dry ingredients:** In a large bowl, whisk together flour, sugar, cocoa powder, baking soda, baking powder, and salt.\n3. **Combine wet ingredients:** In a separate bowl, whisk together buttermilk, oil, eggs, and vanilla extract.\n4. **Add wet to dry:** Gradually add the wet ingredients 